# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [27]:
!python create_tables.py

In [28]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [29]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb")
cur = conn.cursor()

In [30]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [31]:
song_files = get_files('data/song_data')

In [32]:
filepath = song_files[0]

In [33]:
data = pd.read_json(filepath, typ = 'series')
df = pd.DataFrame([data])

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [34]:
song_extracted_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]

In [35]:
song_data = list(song_extracted_data.values[0])
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [36]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [37]:
artist_extracted_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]

In [38]:
artist_data = list(artist_extracted_data.values[0])
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [39]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [40]:
log_files = get_files('data/log_data')

In [41]:
filepath = log_files[0]

In [42]:
df = pd.read_json(filepath, lines = True)
df.head(3)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [43]:
df = df[df['page'] == 'NextSong']

In [ ]:
t 

In [50]:
t = pd.to_datetime(df['ts'], unit = 'ms')

In [52]:
timestamp = list(t.dt.time.values)
hour = list(t.dt.hour.values)
day = list(t.dt.day.values)
week = list(t.dt.isocalendar().week.values)
month = list(t.dt.month.values)
year = list(t.dt.year.values)
weekday = list(t.dt.day_name().values)

In [48]:
list(t.dt.year.values);

In [19]:
df['year'] = df['ts_dt'].dt.year
df['month'] = df['ts_dt'].dt.month
df['week'] = df['ts_dt'].dt.isocalendar().week
df['day'] = df['ts_dt'].dt.day
df['day_of_week'] = df['ts_dt'].dt.day_name()
df['timestamp'] = df['ts_dt']
df['hour'] = df['ts_dt'].dt.hour

In [20]:
extracted_time = df[['timestamp', 'hour', 'day', 'week', 'month', 'year', 'day_of_week']]

time_data = list(df['month'].values), list(df['year'].values), (df['day_of_week'].values)]
column_labels = list(extracted_time.columns)

time_dict = {}
for i in range(0, len(time_data)):
    time_dict[column_labels[i]] = time_data[i]
    
time_df = pd.DataFrame(time_dict)
time_df.head(3)

,timestamp,hour,day,week,month,year,day_of_week
0,02:33:56.796000,2,11,45,11,2018,Sunday
1,02:36:10.796000,2,11,45,11,2018,Sunday
2,02:40:34.796000,2,11,45,11,2018,Sunday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head(3)

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = (row.timestamp, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent )
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [53]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.